In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('data/train.csv', usecols=['Semana','Cliente_ID', 'Producto_ID', 'Demanda_uni_equil'])

In [ ]:
train_grouped = train.groupby(['Cliente_ID','Producto_ID'], as_index=False).sum()

In [ ]:
train_grouped.head()

### Calculations of average demanda (over 7 weeks) for each pair of Cliente and Producto

In [ ]:
train_grouped.Demanda_uni_equil = train_grouped.Demanda_uni_equil/7

In [ ]:
train_grouped.head()

#### Save data of cliente, producto and avg demand to a CSV file, ~ 800 MB

In [ ]:
train_grouped.to_csv('data/train_grouped_ClientProduct_avg.csv')
train_grouped.to_pickle('data/train_grouped_ClientProduct_avg.pkl')

In [ ]:
train_grouped.shape

##### Loading the cpd file (stands for cliente_producto_demanda relationship)

In [2]:
cpd = pd.read_csv('data/train_grouped_ClientProduct_avg.csv')

In [3]:
print ('In cpd.csv, there are: ')
for col in cpd.columns:
     print ('{:>10}{:>20}'.format(len(cpd[col].value_counts()), col))

In cpd.csv, there are: 
  25831538          Unnamed: 0
    880604          Cliente_ID
      1799         Producto_ID
       433              Semana
      4651   Demanda_uni_equil


In [4]:
cpd.shape

(25831538, 5)

In [5]:
cpd.head()

,Unnamed: 0,Cliente_ID,Producto_ID,Semana,Demanda_uni_equil
0,0,26,1182,18,5.571429
1,1,26,4767,18,18.000000
2,2,26,30235,9,13.714286
3,3,26,30314,7,6.857143
4,4,26,31393,42,17.000000


In [6]:
test = pd.read_csv('data/test.csv')

In [7]:
test.shape

(6999251, 7)

In [8]:
print ('In test.csv, there are: ')
for col in test.columns:
     print ('{:>10}{:>20}'.format(len(test[col].value_counts()), col))

In test.csv, there are: 
   6999251                  id
         2              Semana
       552          Agencia_ID
         9            Canal_ID
      2608            Ruta_SAK
    745164          Cliente_ID
      1522         Producto_ID


In [9]:
train_prod_unique = set(cpd['Producto_ID'].unique())
train_client_unique = set(cpd['Cliente_ID'].unique())
test_prod_unique = set(test['Producto_ID'].unique())
test_client_unique = set(test['Cliente_ID'].unique())

In [10]:
print(len(test_prod_unique.difference(train_prod_unique)))
print(len(test_client_unique.difference(train_client_unique)))

34
9663


In [11]:
prod_dict = {}
for pid, p in enumerate(train_prod_unique):
        prod_dict[p] = pid

print(len(prod_dict))

client_dict = {}
for cid, c in enumerate(train_client_unique):
        client_dict[c] = cid

print(len(client_dict))

1799
880604


In [12]:
import numpy as np

In [23]:
two_clients = {}
x  = 0
for c in client_dict:
    x += 1
    if x > 2:
        break
    two_clients[c] = client_dict[c]
print(two_clients)

{2097152: 0, 4194304: 1}


In [37]:
client_prod_matrix = np.zeros((len(train_client_unique), len(train_prod_unique)))
# client_prod_matrix = np.zeros((1000000, 1000000))
# client_prod_matrix = np.zeros((2, len(train_prod_unique)))


for df_idx, row in cpd.iterrows():
#     if df_idx > 1000000:
#         break
    if df_idx % 1000000 == 0:
        print(df_idx)
    m = client_dict[row['Cliente_ID']]
#     if row['Cliente_ID'] not in two_clients:
#         continue
#     m = two_clients[row['Cliente_ID']]
    n = prod_dict[row['Producto_ID']]
    client_prod_matrix[m,n] = row['Demanda_uni_equil']

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000


In [26]:
client_prod_matrix.shape

(2, 1799)

In [38]:
client_prod_matrix.shape

(880604, 1799)

In [39]:
from scipy import sparse

sparse_client_prod_matrix = sparse.csc_matrix(client_prod_matrix)

In [41]:
sparse_client_prod_matrix

<880604x1799 sparse matrix of type '<type 'numpy.float64'>'
	with 25376351 stored elements in Compressed Sparse Column format>

In [28]:
np.save('data/two_client_prod_matrix.pkl', client_prod_matrix)

In [ ]:
np.corrcoef(client_prod_matrix) 

In [42]:
covarian_matrix = np.cov(sparse_client_prod_matrix)
covarian_matrix

ValueError: setting an array element with a sequence.

In [ ]:
np.save('data/matrix.npy',client_prod_matrix )